In [ ]:
%pip install joblib

In [ ]:
import pandas as pd
import numpy as np
import joblib
import load_data as ld
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("new_isc_04.csv")
print(data.shape)
print(data.columns)

(62274, 11)
Index(['id', 'is_tv_subscriber', 'is_movie_package_subscriber',
       'subscription_age', 'bill_avg', 'remaining_contract',
       'service_failure_count', 'download_avg', 'upload_avg',
       'download_over_limit', 'churn'],
      dtype='object')


In [ ]:
# sprawdzenie pustych kolumn dla datasetu:
empty_cells = {column_name: data[column_name].isna().sum() for column_name in data.columns}
print(empty_cells)

{'id': 0, 'is_tv_subscriber': 0, 'is_movie_package_subscriber': 0, 'subscription_age': 0, 'bill_avg': 0, 'remaining_contract': 18524, 'service_failure_count': 0, 'download_avg': 331, 'upload_avg': 331, 'download_over_limit': 0, 'churn': 0}


In [ ]:
# Uzupelnienie pustych wartości zerami
data = data.fillna(0)
empty_cells = {column_name: data[column_name].isna().sum() for column_name in data.columns}
print(empty_cells)

{'id': 0, 'is_tv_subscriber': 0, 'is_movie_package_subscriber': 0, 'subscription_age': 0, 'bill_avg': 0, 'remaining_contract': 0, 'service_failure_count': 0, 'download_avg': 0, 'upload_avg': 0, 'download_over_limit': 0, 'churn': 0}


In [ ]:
# podział datasetu
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['churn']), data['churn'], test_size=0.2, random_state=42)

In [ ]:
# skalowanie danych i zapis wyuczonego skalera

binaries = ['is_tv_subscriber', 'is_movie_package_subscriber']
numbers = [
            'id', 'subscription_age', 'bill_avg', 'remaining_contract',
            'service_failure_count', 'download_avg','upload_avg',
            'download_over_limit'
            ]

# Tworzenie skalera tylko dla numerycznych kolumn
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numbers])
X_test_num = scaler.transform(X_test[numbers])

# Połączenie przeskalowanych numerycznych danych z binarnymi danymi
X_train_scaled = np.hstack((X_train[binaries].values, X_train_num))
X_test_scaled = np.hstack((X_test[binaries].values, X_test_num))

joblib.dump(scaler, 'scaler.sav')



['scaler.sav']

In [ ]:
def count_zeros_ones(y_predict):
    predict_zeros = np.sum(y_predict == 0)
    predict_ones = np.sum(y_predict == 1)


    print(f"y: 0 = {predict_zeros}, 1 = {predict_ones}")


In [ ]:
# Utworzenie modelu Random Forest i zapisanie wyuczonego modelu
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
class_report_rf = classification_report(y_test, y_pred_rf)

print("Random Forest Model")
print(f"Accuracy: {accuracy_rf}")
print("\nConfusion Matrix:")
print(conf_matrix_rf)
print("\nClassification Report:")
print(class_report_rf)
print(count_zeros_ones(y_pred_rf))
print(count_zeros_ones(y_test))

joblib.dump(rf_model, 'random_forest_model.sav')


Random Forest Model
Accuracy: 0.9694098755519871

Confusion Matrix:
[[5437  204]
 [ 177 6637]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      5641
           1       0.97      0.97      0.97      6814

    accuracy                           0.97     12455
   macro avg       0.97      0.97      0.97     12455
weighted avg       0.97      0.97      0.97     12455

y: 0 = 5614, 1 = 6841
None
y: 0 = 5641, 1 = 6814
None


['random_forest_model.sav']

In [ ]:
# przewidywanie dla sekretnej próbki secret_isc_04.csv
new_X = pd.read_csv("secret_isc_04.csv")
print(new_X.shape)
print(new_X.columns)

# sprawdzenie pustych kolumn dla datasetu:
empty_cells = {column_name: new_X[column_name].isna().sum() for column_name in new_X.columns}
print(empty_cells)
# Uzupelnienie pustych wartości zerami
new_X = new_X.fillna(0)
empty_cells = {column_name: new_X[column_name].isna().sum() for column_name in new_X.columns}
print(empty_cells)


(10000, 11)
Index(['id', 'is_tv_subscriber', 'is_movie_package_subscriber',
       'subscription_age', 'bill_avg', 'remaining_contract',
       'service_failure_count', 'download_avg', 'upload_avg',
       'download_over_limit', 'churn'],
      dtype='object')
{'id': 0, 'is_tv_subscriber': 0, 'is_movie_package_subscriber': 0, 'subscription_age': 0, 'bill_avg': 0, 'remaining_contract': 3048, 'service_failure_count': 0, 'download_avg': 50, 'upload_avg': 50, 'download_over_limit': 0, 'churn': 0}
{'id': 0, 'is_tv_subscriber': 0, 'is_movie_package_subscriber': 0, 'subscription_age': 0, 'bill_avg': 0, 'remaining_contract': 0, 'service_failure_count': 0, 'download_avg': 0, 'upload_avg': 0, 'download_over_limit': 0, 'churn': 0}


In [ ]:
loaded_model = joblib.load('random_forest_model.sav')
loaded_scaler = joblib.load('scaler.sav')


binaries = ['is_tv_subscriber', 'is_movie_package_subscriber']
numbers = [
            'id', 'subscription_age', 'bill_avg', 'remaining_contract',
            'service_failure_count', 'download_avg','upload_avg',
            'download_over_limit'
            ]


# # Przewidywanie na nowych danych
new_X_bin = new_X[binaries].values
new_X_num = loaded_scaler.transform(new_X[numbers])
# Połączenie przeskalowanych numerycznych danych z binarnymi danymi
new_X_scaled = np.hstack((new_X_bin, new_X_num))
y_pred_loaded = loaded_model.predict(new_X_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print(f"przewidywanie odejscie klienta {y_pred_loaded}")
print(count_zeros_ones(y_pred_loaded))

przewidywanie odejscie klienta [1 1 1 ... 1 1 1]
y: 0 = 4, 1 = 9996
None


In [ ]:
\# przewidywanie zachowania  dla pojedynczego klienta
loaded_model = joblib.load('random_forest_model.sav')
loaded_scaler = joblib.load('scaler.sav')
new_X = ld.get_user_data()

binaries = ['is_tv_subscriber', 'is_movie_package_subscriber']
numbers = [
            'id', 'subscription_age', 'bill_avg', 'remaining_contract',
            'service_failure_count', 'download_avg','upload_avg',
            'download_over_limit'
            ]


# # Przewidywanie na nowych danych
new_X_bin = new_X[binaries].values
new_X_num = loaded_scaler.transform(new_X[numbers])
new_X_scaled = np.hstack((new_X_bin, new_X_num))
y_pred_loaded = loaded_model.predict(new_X_scaled)


Podaj id klienta: 10
Czy klient posiada tv? Yes or No: yes
Czy klient posiada pakiet wideo? Yes or No: no
Jak długo klient posiada subskrypcję (w miesiącach)? 36
Średnia miesięczna ilość opłaty (w PLN): 450
Ile czasu pozostało umowy (w miesiącach)? 36
Ile razy zgłaszał problem z serwisem? 0
Średnia miesięczna ilość pobranych danych (w GB): 1000
Średnia miesięczna ilość wysłanych danych (w GB): 500
Ile razy klient przekroczył przyznany limit danych? 4


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print(f"przewidywanie odejscie klienta {y_pred_loaded}")

przewidywanie odejscie klienta [0]
